In [1]:
import numpy as np
import sympy as sp
import numba

# Oprações Matriciais

In [55]:
mat33 = np.array([[1,2,3], [5,6,3], [4,8,1]])
mat43 = np.array([[1,2,3], [5,6,3], [4,8,1], [4,-1,10]])
arr = np.array([3,4,5])

In [4]:
mat33

array([[1, 2, 3],
       [5, 6, 3],
       [4, 8, 1]])

## Traço de Matrix

In [6]:
def matrix_trace(a):
    assert a.shape[0] == a.shape[1], 'Matrix must be quadratic'
    sum = 0
    for i in range(len(a)):
        sum += a[i][i]
    
    return sum

In [7]:
matrix_trace(mat33)

8

In [8]:
mat33.trace()

8

## Transposto de Matrix

In [30]:
def matrix_tranpose(a):
    res = np.empty((a.shape[1], a.shape[0]))
    for idx, row in enumerate(a):
        for i, element in enumerate(row):
            res[i, idx] = element

    return res

In [37]:
matrix_tranpose(mat43)

array([[ 1.,  5.,  4.,  4.],
       [ 2.,  6.,  8., -1.],
       [ 3.,  3.,  1., 10.]])

In [38]:
mat43.T

array([[ 1,  5,  4,  4],
       [ 2,  6,  8, -1],
       [ 3,  3,  1, 10]])

## Produto Matricial

In [25]:
@numba.njit()
def matrix_prod(a, b):
    assert a.shape[1] == b.shape[0], "The product is not defined"
    res = np.empty((a.shape[0], b.shape[1]))
    for idx_r, row in enumerate(a):
        for idx_c, col in enumerate(b.T):
            sum = 0
            for i, j in zip(row, col):
                sum += i*j
            res[idx_r, idx_c] = sum
    
    return res


In [28]:
%timeit matrix_prod(mat33, mat33)

652 ns ± 15.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [22]:
%timeit mat33.dot(mat33)

554 ns ± 12.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


# Representando Equações Algébricas Lineares na Forma Matricial

Um sistema de equações lineares pode ser representado como um produto matricual. Vamos ver atraves de um exemplo. 

O sistema abaixo
$$
2x + 3y - z = 2 \\
x - 4y + 2z = 5 \\
-3x - y + 9z = -3
$$
Pode ser escrito como

$$
A = \begin{bmatrix}
2 & 3 & -1 \\
1 & -4 & 2 \\
-3 & -1 & 9 
\end{bmatrix}  
\\
X = \begin{bmatrix}
x  \\
y  \\
z  
\end{bmatrix}
\\
B = \begin{bmatrix}
2  \\
5  \\
-3  
\end{bmatrix}
$$

Aí, com essas definições podem reescrever o sistema na forma matricial
$$
\Rightarrow AX = B
$$ 
Agora, se multiplicarmos os dois lados por $A^{-1}$ teremos
$$
A^{-1} A X = A^{-1} B \rightarrow X = A^{-1} B
$$
Então, a equação foi resolvida e $X$ for determinado. Esse é um outro exemplo de como
a inversa desempenha um papel na álgebra de matrizes que é semelhante à divisão. Devemos observar que esse não é um método muito eficiente para resolver um sistema de equações. Portanto, outras abordagens são empregadas nos algoritmos numéricos.

# Eliminação de Gauss

Este metodo envolve combinar equações para eliminar variáveis. Apesar de ser um dos métodos mais antigos para resolver equações simultâneas, mantém-se como um dos algoritmos mais importantes em uso hoje em dia e é a base da resolução de equações lineares em muitos pacotes de software populares.


## Determinantes e a Regra de Cramer

Determinante é um conceito muito importante em calculo matricial. Vamos ver como esse conceito nos ajuda a descobrir se um sistema tem solução ou não. E alem disso, precisaremos dele para implementar a regra de Cramer para resolver um sistema de eqs. lineares.

O determinante de uma matrix 2x2 é definido como
$$
D = \begin{vmatrix}
a_{11} & a_{12} \\
a_{21} & a_{22}  
\end{vmatrix} 
 = a_{11}a_{22} - a_{12}a_{21}
$$
E para uma matrix 3x3 é 
$$
D = \begin{vmatrix}
a_{11} & a_{12} & a_{13}\\
a_{21} & a_{22} & a_{23} \\
a_{31} & a_{32} & a_{33} 
\end{vmatrix} 
 = 
 a_{11}\begin{vmatrix}
a_{22} & a_{23} \\
a_{32} & a_{33}  
\end{vmatrix}  - 
a_{12}\begin{vmatrix}
a_{21} & a_{23} \\
a_{31} & a_{33}  
\end{vmatrix} + 
a_{13}\begin{vmatrix}
a_{21} & a_{22} \\
a_{31} & a_{32}  
\end{vmatrix}
$$






In [42]:
def matrix_determinent(a):
    assert a.shape[0] == a.shape[1], 'Matrix must be quadratic'
    assert len(a) == 2 or len(a) == 3, 'The shape of matrix must be 2x2 or 3x3'

    if len(a) == 2:
        return a[0,0]*a[1,1] - a[0,1]*a[1,0]
    if len(a) == 3:
        return a[0,0]*matrix_determinent(a[1:,[1,2]]) - a[0,1]*matrix_determinent(a[1:,[0,2]])+\
            a[0,2]*matrix_determinent(a[1:,[0,1]])


In [43]:
matrix_determinent(mat33)

44

In [36]:
np.linalg.det(mat33)

43.99999999999997

Um sistema se diz **singular** se tiver determinante nulo. Neste caso, o sistema não tem uma solução unica. 

## Regra de Cramer. 
Essa regra estabelece que cada incógnita em um sistema de equações algébricas lineares pode ser expressa como uma fração de dois determinantes, com denominador $D$ e com o numerador obtido a partir de $D$ trocando-se a coluna de coeficientes da incógnita em questão pelas constantes $b_1, b_2,..., b_n$ . Por exemplo, $x_1$ é calculada por
$$
x_1 = \frac{\begin{vmatrix}
b_{1} & a_{12} & a_{13}\\
b_{2} & a_{22} & a_{23} \\
b_{3} & a_{32} & a_{33} 
\end{vmatrix} }{D}
$$

### **Exemplo:** Use a regra de Cramer para resolver

$$
\begin{align}
0.3 x_1 &+ 0.52 x_2 + x_3 = -0.01 \\
0.5 x_1 &+ x_2 + 1.9 x_3 = 0.67 \\
0.1 x_1 &+ 0.3 x_2 + 0.5 x_3 = -0.44
\end{align}
$$



In [74]:
def cramer(a, b):
    assert a.shape[0] == a.shape[1], 'Matrix must be quadratic'
    assert a.shape[0] == b.shape[0], 'The dimension of the matrix must the same as the vector of constants'
    D = matrix_determinent(a)
    temp = a.copy()
    temp[:,0] = b
    x1 = matrix_determinent(temp)/D

    temp = a.copy()
    temp[:,1] = b
    x2 = matrix_determinent(temp)/D

    temp = a.copy()
    temp[:,2] = b
    x3 = matrix_determinent(temp)/D

    return x1, x2, x3


In [75]:
A = np.array([[0.3, 0.52, 1], [0.5, 1, 1.9], [0.1, 0.3, 0.5]])
B = np.array([-0.01, 0.67, -0.44])

In [76]:
cramer(A,B)

(-14.900000000000057, -29.500000000000068, 19.80000000000005)

Para mais de três equações, a regra de Cramer torna-se impraticável, pois à medida
que o número de equações cresce, os determinantes demoram mais para ser calculados à
mão (ou pelo computador). Conseqüentemente, são usadas alternativas mais eficientes.

## Eliminação de Gauss Ingênua

Esse método é baseado tecnicas sistematicas para eliminação as variaveis de um sistema progressivamente e substituição regressiva. Apesar de essas técnicas serem hipoteticamente adequadas para implementação em computadores, algumas modificações serãon ecessárias para se obter um algoritmo confiável. Em particular, o programa de computador deve evitar a divisão por zero. O método a seguir é chamado de eliminação de Gauss “ingênua” porque não evita esse problema. Seções subseqüentes irão tratar das características adicionais necessárias para um programa de computador efetivo.
Vamos ver esse método resolvendo o exemplo anterior

$\begin{align*}
3 x_1 - 0.1 x_2 - 0.2 x_3 &= 7.85 \\
0.1 x_1 + 7x_2 - 0.3 x_3 &= -19.3 \\
0.3 x_1 - 0.2 x_2 + 10 x_3 &= 71.4
\end{align*}$

A primeira fase é projetada para reduzir o conjunto de equações a um sistema triangular superior. Para fazer isso vamos eliminar a variavel $x_1$ da segunda equação e $x_1, x_2$ da terceira equação. 

Multiplicamos a primeira linha por $\frac{0.1}{3}$ e depois subtraimos a segunda linha da primeira. O resultado é a nova segunda linha

$$
\begin{align}
\text{Nova segunda linha}&: (0.1 x_1 + 7x_2 - 0.3 x_3 = -19.3) \\ 
& - \frac{0.1}{3}\times(3 x_1 - 0.1 x_2 - 0.2 x_3 = 7.85) \\
&= 7.00333 x_2 - 0.293333 x_3 = -19.5617
\end{align}
$$
Depois dessa operação o conjunto de equações é 
$$
\begin{align}
3 x_1 - 0.1 x_2 - 0.2 x_3 &= 7.85 \\
7.00333 x_2 - 0.293333 x_3 &= -19.5617 \\
0.3 x_1 - 0.2 x_2 + 10 x_3 &= 71.4
\end{align}
$$
Na proxima etapa eliminamos $x_1$ da terceira equação. Para fazer isso multiplicamos a primeira linha por $\frac{0.3}{3}$ e subtraimos da terceira linha. O resultado fica
 $$
\begin{align}
3 x_1 - 0.1 x_2 - 0.2 x_3 &= 7.85 \\
7.00333 x_2 - 0.293333 x_3 &= -19.5617 \\
- 0.1900 x_2 + 10.0200 x_3 &= 70.6150
\end{align}
$$
O ultimo passo da primeira fase é eliminar $x_2$ da terceira equação. Multiplicamos a segunda equação por $\frac{-.1900}{7.0033}$ e subtraimos da terceira. Isso elimina $x_2$ da terceira equação e reduz o sistema a um sistema triangular superior, como em
 $$
\begin{align}
3 x_1 - 0.1 x_2 - 0.2 x_3 &= 7.85 \\
7.00333 x_2 - 0.293333 x_3 &= -19.5617 \\
 10.120 x_3 &= 70.0843
\end{align}
$$
Agora, é possível resolver essas equações por substituição regressiva. Primeiro, a
terceira equação pode ser resolvida para determinar
$$
x_3 = \frac{70.843}{10.0120} = 7.000
$$
Esse resultado pode ser substituído na segunda equação
$$
7.0033 x_2 - 0.293333(7.000) = -2.5000
$$
Finalmente os valores de $x_3$ e $x_2$ podem ser inseridos na primeira equação
$$
3x_1 - 0.1(-2.5000) - 0.2(7.0000) = 7.85 = 3.000
$$
Tais resultados são idênticos à solução exata. Podemos testar o resultado inserindo os valores no sistema.

Esse metodo é facilmente estendido para uma sistema com $n$ equações e $n$ variaveis. 

Para uma sistema com $n$ variaveis o numero das operações com ponto flutuante (ou flops) para resolver o sistema usando o metodo de eliminação de Gauss é da ordem 
$$
 \underbrace{\frac{2n^3}{2} +\mathcal{O}(n^2)}_{\text{Eliminação progressiva}} +  \underbrace{n^2 +\mathcal{O}(n)}_{\text{Substituição regressiva}} \overbrace{\longrightarrow}^{\text{quando n cresce}} \frac{2n^3}{2} +\mathcal{O}(n^2)
$$
Agora vamos tentar implementar este algoritmo. O pseducorde para este algoritmo é 

<img src="./images/gauss_elimi_simple.jpg" style="width:250px;height:350px;">

In [140]:
def gauss_elimination_minimal(A, b):
    assert A.shape[0] == A.shape[1], 'the matrix of coefficients must be squared'

    mat = np.concatenate([A, b.reshape(-1,1)], axis=1)
    for idx_i in range(0, len(mat)-1):
        for idx_j in range(idx_i+1, len(mat)):
            factor = mat[idx_j, idx_i] / mat[idx_i, idx_i] 
            mat[idx_j] = mat[idx_j] - factor * mat[idx_i]

    n = len(A)-1
    xs = np.zeros(len(A))
    xs[-1] = mat[n,n+1]/mat[n,n]

    for i in range(n-1, -1, -1):
        sum = mat[i, -1]
        for j in range(0, len(A)):
            sum -= mat[i, j]* xs[j]
        xs[i] = sum / mat[i,i]

    return xs

In [141]:
AA = np.array([[3, -.1, -0.2], [0.1, 7, -0.3], [0.3, -0.2, 10]])
bb = np.array([7.85, -19.3, 71.4])

In [142]:
gauss_elimination_minimal(AA, bb)

array([ 3. , -2.5,  7. ])

## Armadilhas dos métodos de eliminação

Embora existam muitos sistemas de equações que podem ser resolvidas pela eliminação
de Gauss ingênua, há algumas armadilhas que precisam ser exploradas antes de escrever
um programa de computador geral para implementar o método.

### Divisão por zero

A principal razão para que a técnica anterior seja chamada de “ingênua” é que tanto durante a fase de eliminação quanto durante a de substituição é possível ocorrer uma divisão por zero. Por exemplo, se for usada a eliminação de Gauss ingênua para resolver

$$
\begin{align}
2 x_2 - 3 x_3 &= 7 \\
4x_1 + 6 x_2 - 7 x_3 &= -3 \\
2 x_1 -  x_2 + 6 x_3 &= 5
\end{align}
$$

a normalização da primeira linha iria envolver a divisão por $a_{11} = 0$. Também podem surgir problemas quando um coeficiente está muito próximo de zero. A técnica de pivotamento foi desenvolvida para evitar parcialmente esses problemas.

### Erros de Arredondamento